In [1]:
import sys
sys.path.append('../')

from magpie import grasp as gt
import rtde_control
import rtde_receive
from magpie.motor_code import Motors
from magpie import ur5 as ur5
import time
import numpy as np
import copy
import magpie
from magpie.gripper import Gripper
from magpie import poses

In [2]:
from magpie import realsense_wrapper
import asyncio
from concurrent.futures import ThreadPoolExecutor

devices = realsense_wrapper.poll_devices()
print(devices)
rsc = realsense_wrapper.RealSense()
rsc.initConnection(device_serial=devices['D405'])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
{'D435': '926522071700', 'D405': '126122270157'}


In [3]:
rsc.getPCD()

(PointCloud with 0 points.,
 RGBDImage of size 
 Color image : 1280x720, with 3 channels.
 Depth image : 1280x720, with 1 channels.
 Use numpy.asarray to access buffer data.)

In [4]:
devices = realsense_wrapper.poll_devices()
workspace_rs = realsense_wrapper.RealSense(zMax=5, fps=15)
# workspace_rs.initConnection(device_serial=devices['D435'], enable_depth=False)
workspace_rs.initConnection(device_serial=devices['D435'])

In [2]:
import importlib
importlib.reload(ur5)

<module 'magpie.ur5' from '/home/will/MAGPIE/tests/../magpie/ur5.py'>

In [5]:
tmat = np.array([[ 0.50676157,  0.63485957,  0.58322041,  0.03397274],
        [ 0.46190666, -0.77117344,  0.43810246, -0.04689727],
        [-0.72789762, -0.0473799 ,  0.68404692,  0.37931564],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])

z_offset = 0.10
# z_offset += -0.045

# eye = np.eye(4)
# eye[2, 3] = -z_offset

# tmat = tmat @ eye
print(tmat)
# aperture = 37
# # z_offset = G.aperture_to_z(aperture)/1000.0

# mmc = copy.deepcopy(tmat[:3, 3])
# gripper_offset = [0.012, 0.006, 0.221] # x, y, z offset
# grasp_pose = mmc - gripper_offset

# # make identity matrix with grasp_pose as translation
# tmat_offset = np.eye(4)
# tmat_offset[:3, 3] = grasp_pose
# print(tmat_offset)

# tmat[:3, 3] = [0, 0, 0]
homePose = None

[[ 0.507  0.635  0.583  0.034]
 [ 0.462 -0.771  0.438 -0.047]
 [-0.728 -0.047  0.684  0.379]
 [ 0.     0.     0.     1.   ]]


In [8]:
def robot_motion():
    sleepRate = 1.5
    # time.sleep(sleepRate)
    ur = ur5.UR5_Interface()
    try:
        ur.start()
        homePose = ur.get_tcp_pose()
        ur.move_tcp_cartesian(tmat)
        time.sleep(sleepRate * 1.2)# offset_pose = gt.get_world_frame(mmc, ur, -offset)
        ur.stop()
    except Exception as e:
        ur.stop()
        raise(e)


In [9]:
cp_dict = {rsc: 'record/wrist/', workspace_rs: 'record/workspace/'}
async def main():

    # rsc.begin_record(filepath='record/wrist/')
    # workspace_rs.begin_record(filepath='record/workspace/')
    ur = ur5.UR5_Interface()
    ur.start()
    for camera in cp_dict:
        camera.begin_record(filepath=cp_dict[camera])
    await asyncio.sleep(1)
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as pool:
        # await loop.run_in_executor(pool, robot_motion)
        await loop.run_in_executor(pool, ur.move_tcp_cartesian, tmat)
        await asyncio.sleep(0.5 * 3)
    for camera in cp_dict:
        await camera.stop_record()
    ur.stop_recording()
    ur.stop()
    # await rsc.stop_record()
    # await workspace_rs.stop_record()


await main()

Succeeded to open the port
Succeeded to change the baudrate
Recording Started
Recording Started
Stopping Recording
Recording Task Cancelled
Recording Stopped
Stopping Recording
Recording Task Cancelled
Recording Stopped


In [7]:
rsc.begin_record(filepath='record/')
loop = asyncio.get_event_loop()
with ThreadPoolExecutor() as pool:
    await loop.run_in_executor(pool, robot_motion)
await rsc.stop_record()



Recording Started
Succeeded to open the port
Succeeded to change the baudrate
Stopping Recording
Recording Task Cancelled
Recording Stopped


In [16]:
ur = ur5.UR5_Interface()
try:
    ur.start()
    ur.close_gripper()
    # ur.open_gripper()
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate


In [12]:
# move home
ur = ur5.UR5_Interface()
try:
    ur.start()
    # ur.open_gripper()
    # time.sleep(sleepRate)
    # homePose = np.array([[-0.024, -0.998, -0.062, -0.261],
    #                     [-0.999,  0.021,  0.035, -0.162],
    #                     [-0.033,  0.063, -0.997,  0.221],
    #                     [ 0.   ,  0.   ,  0.   ,  1.   ]])
    ur.moveL(homePose)
    # gt_home = ur.getPose()
    # print(np.array(gt_home))
    time.sleep(sleepRate * 1.1)
    ur.stop()
except Exception as e:
    ur.stop()

Succeeded to open the port
Succeeded to change the baudrate


In [4]:
# # move gripper to cartesian pos
# # OR orient gripper in place
# sleepRate = 1.5
# # time.sleep(sleepRate)
# ur = ur5.UR5_Interface()
# try:
#     ur.start()
#     homePose = ur.get_tcp_pose()
#     # homePose = ur.getPose()
#     # print("Home pose: ", homePose)
    
#     # in-place (or best attempt) re-orientation
#     # currentPose = ur.getPose()
#     # desiredPose = np.matmul(np.array(currentPose), tmat)
#     # ur.moveL(desiredPose)
#     # time.sleep(sleepRate * 1.2)

#     ## move to cartesian pos
#     gt.move_to_L(grasp_pose, ur, z_offset=z_offset)    
#     print("Done moving to block")
#     time.sleep(sleepRate * 1.2)
#     ur.close_gripper()

#     ur.stop()
# except Exception as e:
#     ur.stop()
#     raise(e)

Succeeded to open the port
Succeeded to change the baudrate
[TxRxResult] Incorrect status packet!
[TxRxResult] There is no status packet!
[TxRxResult] There is no status packet!
[TxRxResult] There is no status packet!
[[ 1.     0.004 -0.004  0.077]
 [ 0.004 -1.     0.022 -0.318]
 [-0.004 -0.022 -1.     0.089]
 [ 0.     0.     0.     1.   ]]
P_goal:
[-0.008  0.006 -0.09 ]
Done moving to block
